<a href="https://colab.research.google.com/github/Saylion/SongInverence/blob/main/Song_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<font face="verdana"><font size="8"<strong>SELAMAT DATANG DI SONG INFERENCE COLAB</strong></font>
<font face="verdana"><p align="center"><i>Last update at : 26 Maret 2024</i></p>
<img src="https://www.gambaranimasi.org/data/media/562/animasi-bergerak-garis-0091.gif" border="0" height="8" width="850">
<br>
<br>

Jika ada saran atau error, bisa langsung menghubungi email berikut
<a href="mailto:iacorporation.business@gmail.com" alt="my email">email me</a>

<i>If you have any suggestions or errors,  you can directly contact the following email
<a href="mailto:iacorporation.business@gmail.com" alt="my email">email me</a></i>
</center>

<hr>
<br>
<p>Changelog (26 Maret 2024):</p>
<ol>
  <li>Menambahkan fitur metode pemisahan vocal dan instrumen menggunakan Demucs/UVR-MDXNET<br>
    <i>"Added vocal and instrument separation method feature using Demucs/UVR-MDXNET"</i>
  </li><br>
  <li>Perbaikan beberapa bugs<br>
    <i>"Fixed some bugs</i>
  </li>
</ol>

<p>More about <a href="https://github.com/Saylion/SongInverence/blob/d0c0562fc5b0a3970ba243e0bb4d0119f2d46fe8/changelog.txt">changelog</a><br></p><br>

Google colab collection :
<ul>
  <li>RVC Mode Training (include inference)</li>

<a href="https://colab.research.google.com/drive/1-efktIIzxUMu6NqhBX6H57TEseMY3Zee?usp=drive_link"><img src="https://colab.research.google.com/assets/colab-badge.svg"></a>
</ul>
<br><br><br><br>

In [ ]:
#@title Input Form
#@markdown Fill modelname to inference <br>
#@markdown (bisa multi model, pisahkan dengan spasi atau pilih All model untuk memindahkan semua model dari drive)

Input_Option = "Manual" #@param["Manual", "All Model"]

Option = Input_Option

MODELNAME = "Your_model"  #@param {type:"string"}

Save_output_song_to_drive = False #@param{type:'boolean'}

Save = Save_output_song_to_drive

import sys
import time
!pip install colorama &> /dev/null
from colorama import Fore, Back, Style

if MODELNAME == "Your_model" and Option == "Manual":
   print("\n")
   print(Back.RED, Fore.WHITE + "\033[1mMasukkan nama model dengan benar atau pilih All Model!\033[0m","\n" + Style.RESET_ALL)
   print("Nama model", "\033[1m", MODELNAME, "\033[0madalah template\n")
   print("Menghentikan runtime...\n")
   time.sleep(3)
   print("Runtime Berhenti!")

   exit()

In [ ]:
#@title Mount Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Clone repository
from IPython.display import clear_output, Javascript
!git clone https://github.com/Saylion/SongInverence.git
clear_output()
print("Proses cloning repository selesai, lanjut ke sel selanjutnya")

In [ ]:
#@title Fix CUDA Problem
!sudo apt update
!yes | sudo DEBIAN_FRONTEND=noninteractive apt-get -yq install cuda-11-8
!rm -rf sample_data
clear_output()
print("Proses Selesai")
%cd SongInverence

In [ ]:
#@title Install requirements
!pip install -q -r requirements.txt
!pip install gradio-client==0.8.1
!pip install gradio==3.48.0
!pip install torch==2.0.1+cu118 torchaudio==2.0.2+cu118 -f https://download.pytorch.org/whl/torch_stable.html
!python3 -m pip install -U demucs
!sudo apt update
!sudo apt install sox

clear_output()
print("Instalasi kebutuhan selesai, lanjut ke sel berikutnya ")

In [ ]:
#@title Download MDXNet Vocal Separation and Hubert Base Models
!python src/download_models.py

In [ ]:
#@title Copy modelname

import numpy as np
import os

# Create an empty NumPy array
modelnames = np.array([])

# Split the user input into a list of strings
model_input_list = MODELNAME.split(" ")

# Convert the list of strings to a NumPy array
modelnames = np.array(model_input_list)

#List folder RVC di Drive
path = "/content/drive/MyDrive/RVC"

folders = os.listdir(path)
array_folders = []
for folder in folders:
  if not os.path.isfile(os.path.join(path, folder)):
    array_folders.append(folder)

list_folder = np.array(array_folders)

if Option == "Manual":
    for modelname in modelnames:
        !mkdir -p /content/SongInverence/rvc_models/{modelname}/


        !cp /content/drive/MyDrive/RVC/{modelname}/added_*.index /content/SongInverence/rvc_models/{modelname}/

        !cp /content/drive/MyDrive/RVC/weights/{modelname}.pth /content/SongInverence/rvc_models/{modelname}

else:
    for folder in list_folder:
        if not folder == "weights" and not folder == ".ipynb_checkpoints":
           !mkdir -p /content/SongInverence/rvc_models/{folder}/
           !cp /content/drive/MyDrive/RVC/{folder}/added_*.index /content/SongInverence/rvc_models/{folder}/
           !cp -r /content/drive/MyDrive/RVC/weights/{folder}.pth /content/SongInverence/rvc_models/{folder}

clear_output()
# Temukan semua folder yang berisi kurang dari 2 file (tidak termasuk subfolder) dan hapus
%cd /content/SongInverence/rvc_models
!find . -type d -exec bash -c '[[ $(find "$1" -maxdepth 1 -type f | wc -l) -lt 2 ]]' _ {} \; -print0 | xargs -0 rm -rf

length = len(modelnames)
if length:
    print("Model sudah berhasil dipindah ke")
else:
    print("Semua model sudah berhasil dipindah ke")
print("\nList Model Saat Ini:")
!find . -maxdepth 1 -type d | sort -V
print("\n")
%cd /content/SongInverence

In [ ]:
#@title Run WebUI

!python src/webui.py --share

In [ ]:
#@title Copy Output song to drive (Optional)

if Save == True and Option == "Manual":
  for modelname in modelnames:
      !mkdir -p /content/drive/MyDrive/Output/{modelname}
      !cp -r /content/SongInverence/song_output/* /content/drive/MyDrive/Output/{modelname}
      print("output sudah di copy ke drive")
elif Save == True and Option == "All Model":
      !mkdir -p /content/drive/MyDrive/Output/Output_All
      !cp -r /content/SongInverence/song_output/ /content/drive/MyDrive/Output/Output_All
      print("output sudah di copy ke drive")
else:
  print("output tidak di copy ke drive")